In [1]:
using CompTime, InteractiveUtils, MacroTools

In [2]:
cleanup = MacroTools.flatten ∘ Base.remove_linenums!

MacroTools.flatten ∘ Base.remove_linenums!

Polynomial:

$p(x) = a_1 + a_2 x + \cdots + a_n x^{n-1}$


Horner's method:

$p(x) = (((a_n x  + a_{n-1}) x + a_{n-2}) x + \cdots + a_{2})x + a_{1}$


In [3]:
"""
Evaluate a polynomial whose coefficients are given in ascending
order in `c`, at the point `x`, using Horner's rule.
"""

function horner(a, x)
    n = length(a)
    y = a[n]

    for k in n-1:-1:1
        y = x * y + a[k]
    end

    return y
end

horner (generic function with 1 method)

In [4]:
a = [11, 12, 13, 14]

4-element Vector{Int64}:
 11
 12
 13
 14

In [5]:
horner(a, 10)

15431

In [6]:
@code_lowered horner(a, 10)

CodeInfo(
1 ─       n = Main.length(a)
│         y = Base.getindex(a, n)
│   %3  = n - 1
│   %4  = %3:-1:1
│         @_4 = Base.iterate(%4)
│   %6  = @_4 === nothing
│   %7  = Base.not_int(%6)
└──       goto #4 if not %7
2 ┄ %9  = @_4
│         k = Core.getfield(%9, 1)
│   %11 = Core.getfield(%9, 2)
│   %12 = x * y
│   %13 = Base.getindex(a, k)
│         y = %12 + %13
│         @_4 = Base.iterate(%4, %11)
│   %16 = @_4 === nothing
│   %17 = Base.not_int(%16)
└──       goto #4 if not %17
3 ─       goto #2
4 ┄       return y
)

In [7]:
function horner_tuple(c::NTuple{N,T}, x::T) where {N,T}
    y = c[N]

    for k in N-1:-1:1
        y = x * y + c[k]
    end

    return y
end

horner_tuple (generic function with 1 method)

In [8]:
t = (11, 12, 13, 14)

(11, 12, 13, 14)

In [9]:
horner_tuple(t, 10)

15431

In [10]:
@code_lowered horner_tuple(t, 10)

CodeInfo(
1 ─       y = Base.getindex(c, $(Expr(:static_parameter, 1)))
│   %2  = $(Expr(:static_parameter, 1)) - 1
│   %3  = %2:-1:1
│         @_4 = Base.iterate(%3)
│   %5  = @_4 === nothing
│   %6  = Base.not_int(%5)
└──       goto #4 if not %6
2 ┄ %8  = @_4
│         k = Core.getfield(%8, 1)
│   %10 = Core.getfield(%8, 2)
│   %11 = x * y
│   %12 = Base.getindex(c, k)
│         y = %11 + %12
│         @_4 = Base.iterate(%3, %10)
│   %15 = @_4 === nothing
│   %16 = Base.not_int(%15)
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return y
)

In [11]:
@generated function horner_gen_expr(c::NTuple{N,T}, x::T) where {N,T}
    y = :(c[$N])

    for k in N-1:-1:1
        y = :(x * $y + c[$k])
    end

    Core.println(string(y))
    return y
end

horner_gen_expr (generic function with 1 method)

In [12]:
horner_gen_expr(t, 10)

x * (x * (x * c[4] + c[3]) + c[2]) + c[1]


15431

In [13]:
@generated function horner_gen(c::NTuple{N,T}, x::T) where {N,T}
    r = Expr[]
    push!(r, :(y = c[$N]))

    for k in N-1:-1:1
        push!(r, :(y = x * y + c[$k]))
    end
 
    r = :(begin $(r...) end)
    Core.println(string(cleanup(r)))
    return r
end

horner_gen (generic function with 1 method)

In [14]:
horner_gen(t, 10)

begin
    y = c[4]
    y = x * y + c[3]
    y = x * y + c[2]
    y = x * y + c[1]
end


15431

In [15]:
@ct_enable function horner_ct(c::NTuple{N,T}, x::T) where {N,T}
    y = c[@ct(N)]

    @ct_ctrl for k in N-1:-1:1
        y = x * y + c[@ct(k)]
    end

    return y
end

runtime (generic function with 1 method)

In [16]:
horner_ct(t, 10)

15431

In [17]:
debug(horner_ct, t, 10) |> cleanup

quote
    y = c[4]
    y = x * y + c[3]
    y = x * y + c[2]
    y = x * y + c[1]
    return y
end

In [18]:
@generated function horner_gen_expr2(::Val{c}, x) where {c}
    N = length(c)
    r = :($(c[N]))

    for k in N-1:-1:1
        r = :(x * $r + $(c[k]))
    end

    Core.println(string(r))
    return r
end

horner_gen_expr2 (generic function with 1 method)

In [19]:
horner_gen_expr2(Val(t), 10)

x * (x * (x * 14 + 13) + 12) + 11


15431

In [20]:
@generated function horner_gen2(::Val{c}, x) where {c}
    N = length(c)
    r = Expr[]
    push!(r, :(y = $(c[N])))

    for k in N-1:-1:1
        push!(r, :(y = x * y + $(c[k])))
    end
 
    r = :(begin $(r...) end)
    Core.println(string(cleanup(r)))
    return r
end

horner_gen2 (generic function with 1 method)

In [21]:
horner_gen2(Val(t), 10)

begin
    y = 14
    y = x * y + 13
    y = x * y + 12
    y = x * y + 11
end


15431

In [22]:
@ct_enable function horner_ct2(@ct(c), x)
    @ct(N = length(c))
    y = @ct(c[N])

    @ct_ctrl for k in N-1:-1:1
        y = x * y + @ct(c[k])
    end

    return y
end

runtime (generic function with 2 methods)

In [23]:
runtime(horner_ct2, t, 10)

15431

In [24]:
@code_lowered runtime(horner_ct2, t, 10)

CodeInfo(
1 ─       N = Main.length(c)
│         y = Base.getindex(c, N)
│   %3  = N - 1
│   %4  = %3:-1:1
│         @_5 = Base.iterate(%4)
│   %6  = @_5 === nothing
│   %7  = Base.not_int(%6)
└──       goto #4 if not %7
2 ┄ %9  = @_5
│         k = Core.getfield(%9, 1)
│   %11 = Core.getfield(%9, 2)
│   %12 = x * y
│   %13 = Base.getindex(c, k)
│         y = %12 + %13
│         @_5 = Base.iterate(%4, %11)
│   %16 = @_5 === nothing
│   %17 = Base.not_int(%16)
└──       goto #4 if not %17
3 ─       goto #2
4 ┄       return y
)

In [25]:
debug(horner_ct2, Val{t}, 10) |> cleanup

quote
    4
    y = 14
    y = x * y + 13
    y = x * y + 12
    y = x * y + 11
    return y
end